In [0]:
!pip install torch torchvision tqdm
import torch, os
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

    100% |████████████████████████████████| 519.5MB 31kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5968e000 @  0x7fb3ed1f31c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 1.7MB/s 
    100% |████████████████████████████████| 51kB 11.7MB/s 
    100% |████████████████████████████████| 2.0MB 2.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
transform = transforms.Compose([transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [0]:
def train(epoch, trainloader, optimizer, criterion):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader), 0):
        # get the inputs
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('epoch %d training loss: %.3f' %
            (epoch + 1, running_loss / (len(trainloader))))

In [0]:
def test(testloader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
                                    100 * correct / total))

In [0]:
def classwise_test(testloader, model):
########################################################################
# class-wise accuracy

    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))


In [0]:
num_epochs = 30
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=5)#32-5+1=28
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#14
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=5)#14-5+1=10
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.conv3 = nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3)#5-3+1=3
        #self.conv4 = nn.Conv2d(in_channels=192, out_channels=192, kernel_size=1)#1
        #self.conv5 = nn.Conv2d(in_channels=192, out_channels=10, kernel_size=1)#1
        self.fc1 = nn.Linear(in_features=192*3*3, out_features=192)
        self.fc2 = nn.Linear(in_features=192, out_features=10)
        #self.fc3 = nn.Linear(in_features=96, out_features=10)


    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        
        x = x.view(-1, 192*3*3)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')



  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1




  0%|          | 1/12500 [00:00<21:36,  9.64it/s]

  0%|          | 10/12500 [00:00<04:24, 47.16it/s]

  0%|          | 19/12500 [00:00<03:30, 59.15it/s]

  0%|          | 27/12500 [00:00<03:18, 62.70it/s]

  0%|          | 33/12500 [00:00<03:21, 61.87it/s]

  0%|          | 40/12500 [00:00<03:20, 62.24it/s]

  0%|          | 48/12500 [00:00<03:15, 63.82it/s]

  0%|          | 55/12500 [00:00<03:14, 64.12it/s]

  1%|          | 64/12500 [00:00<03:07, 66.45it/s]

  1%|          | 73/12500 [00:01<03:01, 68.58it/s]

  1%|          | 83/12500 [00:01<02:54, 71.11it/s]

  1%|          | 93/12500 [00:01<02:50, 72.94it/s]

  1%|          | 102/12500 [00:01<02:48, 73.73it/s]

  1%|          | 112/12500 [00:01<02:44, 75.12it/s]

  1%|          | 121/12500 [00:01<02:43, 75.73it/s]

  1%|          | 130/12500 [00:01<02:42, 75.98it/s]

  1%|          | 139/12500 [00:01<02:45, 74.90it/s]

  1%|          | 147/12500 [00:02<02:49, 73.07it/s]

  1%|          | 154/12500 [00:02<02:51, 71.90it/s]

  1%

epoch 1 training loss: 1.717




  0%|          | 1/2500 [00:00<04:35,  9.09it/s]

  1%|          | 14/2500 [00:00<00:36, 68.57it/s]

  1%|▏         | 32/2500 [00:00<00:23, 103.64it/s]

  2%|▏         | 50/2500 [00:00<00:20, 121.41it/s]

  3%|▎         | 68/2500 [00:00<00:18, 132.39it/s]

  3%|▎         | 86/2500 [00:00<00:17, 139.39it/s]

  4%|▍         | 104/2500 [00:00<00:16, 144.60it/s]

  5%|▍         | 120/2500 [00:00<00:16, 145.82it/s]

  6%|▌         | 138/2500 [00:00<00:15, 149.24it/s]

  6%|▌         | 156/2500 [00:01<00:15, 151.74it/s]

  7%|▋         | 175/2500 [00:01<00:15, 154.42it/s]

  8%|▊         | 192/2500 [00:01<00:14, 155.46it/s]

  8%|▊         | 209/2500 [00:01<00:14, 156.21it/s]

  9%|▉         | 226/2500 [00:01<00:14, 155.22it/s]

 10%|▉         | 244/2500 [00:01<00:14, 156.35it/s]

 10%|█         | 261/2500 [00:01<00:14, 156.93it/s]

 11%|█         | 279/2500 [00:01<00:14, 157.90it/s]

 12%|█▏        | 296/2500 [00:01<00:13, 158.38it/s]

 13%|█▎        | 314/2500 [00:01<00:13, 159.09it/s]



...



  0%|          | 1/2500 [00:00<04:31,  9.19it/s]

  1%|          | 18/2500 [00:00<00:28, 86.99it/s]

  1%|▏         | 36/2500 [00:00<00:21, 116.83it/s]

  2%|▏         | 54/2500 [00:00<00:18, 132.11it/s]

  3%|▎         | 72/2500 [00:00<00:17, 140.80it/s]

  4%|▎         | 89/2500 [00:00<00:16, 144.88it/s]

  4%|▍         | 105/2500 [00:00<00:16, 146.21it/s]

  5%|▍         | 123/2500 [00:00<00:15, 150.17it/s]

  6%|▌         | 141/2500 [00:00<00:15, 153.19it/s]

  6%|▋         | 158/2500 [00:01<00:15, 154.00it/s]

  7%|▋         | 176/2500 [00:01<00:14, 155.92it/s]

  8%|▊         | 194/2500 [00:01<00:14, 157.53it/s]

  9%|▊         | 213/2500 [00:01<00:14, 159.55it/s]

  9%|▉         | 231/2500 [00:01<00:14, 160.34it/s]

 10%|▉         | 249/2500 [00:01<00:13, 161.23it/s]

 11%|█         | 267/2500 [00:01<00:13, 162.06it/s]

 11%|█▏        | 285/2500 [00:01<00:13, 162.89it/s]

 12%|█▏        | 304/2500 [00:01<00:13, 163.67it/s]

 13%|█▎        | 322/2500 [00:01<00:13, 162.98it/s]



Accuracy of the network on the 10000 test images: 71 %




  0%|          | 2/2500 [00:00<02:14, 18.51it/s]

  1%|          | 18/2500 [00:00<00:29, 85.36it/s]

  1%|▏         | 33/2500 [00:00<00:24, 102.55it/s]

  2%|▏         | 47/2500 [00:00<00:21, 112.69it/s]

  3%|▎         | 63/2500 [00:00<00:20, 121.06it/s]

  3%|▎         | 79/2500 [00:00<00:19, 127.04it/s]

  4%|▍         | 96/2500 [00:00<00:18, 131.97it/s]

  4%|▍         | 112/2500 [00:00<00:17, 134.71it/s]

  5%|▌         | 128/2500 [00:00<00:17, 136.69it/s]

  6%|▌         | 144/2500 [00:01<00:16, 138.81it/s]

  6%|▋         | 160/2500 [00:01<00:16, 140.29it/s]

  7%|▋         | 176/2500 [00:01<00:16, 141.74it/s]

  8%|▊         | 192/2500 [00:01<00:16, 142.60it/s]

  8%|▊         | 208/2500 [00:01<00:16, 143.12it/s]

  9%|▉         | 224/2500 [00:01<00:15, 142.46it/s]

 10%|▉         | 240/2500 [00:01<00:15, 143.12it/s]

 10%|█         | 256/2500 [00:01<00:15, 143.57it/s]

 11%|█         | 273/2500 [00:01<00:15, 144.43it/s]

 12%|█▏        | 289/2500 [00:01<00:15, 145.14it/s]

 

Accuracy of plane : 78 %
Accuracy of   car : 83 %
Accuracy of  bird : 66 %
Accuracy of   cat : 46 %
Accuracy of  deer : 64 %
Accuracy of   dog : 63 %
Accuracy of  frog : 79 %
Accuracy of horse : 76 %
Accuracy of  ship : 82 %
Accuracy of truck : 77 %
epoch  20




  0%|          | 2/12500 [00:00<11:06, 18.75it/s]

  0%|          | 12/12500 [00:00<03:41, 56.31it/s]

  0%|          | 22/12500 [00:00<02:58, 69.82it/s]

  0%|          | 32/12500 [00:00<02:43, 76.06it/s]

  0%|          | 42/12500 [00:00<02:35, 80.11it/s]

  0%|          | 52/12500 [00:00<02:30, 82.85it/s]

  0%|          | 61/12500 [00:00<02:30, 82.39it/s]

  1%|          | 71/12500 [00:00<02:27, 84.37it/s]

  1%|          | 81/12500 [00:00<02:25, 85.62it/s]

  1%|          | 91/12500 [00:01<02:23, 86.66it/s]

  1%|          | 101/12500 [00:01<02:21, 87.55it/s]

  1%|          | 111/12500 [00:01<02:20, 88.02it/s]

  1%|          | 121/12500 [00:01<02:19, 88.47it/s]

  1%|          | 131/12500 [00:01<02:18, 89.01it/s]

  1%|          | 141/12500 [00:01<02:18, 89.46it/s]

  1%|          | 151/12500 [00:01<02:17, 89.86it/s]

  1%|▏         | 161/12500 [00:01<02:16, 90.27it/s]

  1%|▏         | 171/12500 [00:01<02:16, 90.39it/s]

  1%|▏         | 181/12500 [00:02<02:16, 90.01it/s]

  